In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.3 MB/s eta 0:00:00


In [ ]:
pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.18.0
    Uninstalling huggingface-hub-0.18.0:
      Successfully uninstalled huggingface-hub-0.18.0


In [ ]:
from datasets import load_dataset
data = load_dataset("billsum")
data

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 18949
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 3269
    })
    ca_test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 1237
    })
})

In [ ]:
from transformers import AutoTokenizer

kerek = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(kerek)
tokenizer

T5TokenizerFast(name_or_path='t5-small', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_i

In [ ]:
prefix = "summarize: "


def preprocess(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, padding="max_length", truncation=True)

    labels = tokenizer(text_target=examples["summary"], padding="max_length", max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
new_data = data.map(preprocess, batched=True)

Map:   0%|          | 0/18949 [00:00<?, ? examples/s]

Map:   0%|          | 0/3269 [00:00<?, ? examples/s]

Map:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForSeq2Seq, TFAutoModelForSeq2SeqLM

coll = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=kerek, return_tensors="tf")
model = TFAutoModelForSeq2SeqLM.from_pretrained(kerek)

All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
train = model.prepare_tf_dataset(
    new_data["train"],
    shuffle=True,
    batch_size=4,
    collate_fn=coll,
)

test = model.prepare_tf_dataset(
    new_data["test"],
    shuffle=False,
    batch_size=4,
    collate_fn=coll,
)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [ ]:
import tensorflow as tf

model.compile(optimizer=optimizer, metrics=["accuracy"])

In [ ]:
model.fit(x=train, validation_data=test, epochs=1)

4737/4737 [==============================] - 2210s 462ms/step - loss: 1.8984 - accuracy: 0.6449 - val_loss: 1.4788 - val_accuracy: 0.7086


In [ ]:
model.save_pretrained("t5-small-sumarization")

In [ ]:
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [ ]:
from transformers import pipeline

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

sample_text = new_data["test"][0]["text"]

reference = new_data["test"][0]["summary"]

pipe = pipeline("summarization", model="t5-small-sumarization",tokenizer=tokenizer)

##
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small-sumarization.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.
Token indices sequence length is longer than the specified maximum sequence length for this model (1814 > 512). Running this sequence through the model will result in indexing errors


Dialogue:
SECTION 1. ENVIRONMENTAL INFRASTRUCTURE.

    (a) Jackson County, Mississippi.--Section 219 of the Water 
Resources Development Act of 1992 (106 Stat. 4835; 110 Stat. 3757) is 
amended--
        (1) in subsection (c), by striking paragraph (5) and inserting 
    the following:
        ``(5) Jackson county, mississippi.--Provision of an alternative 
    water supply and a project for the elimination or control of 
    combined sewer overflows for Jackson County, Mississippi.''; and
        (2) in subsection (e)(1), by striking ``$10,000,000'' and 
    inserting ``$20,000,000''.
    (b) Manchester, New Hampshire.--Section 219(e)(3) of the Water 
Resources Development Act of 1992 (106 Stat. 4835; 110 Stat. 3757) is 
amended by striking ``$10,000,000'' and inserting ``$20,000,000''.
    (c) Atlanta, Georgia.--Section 219(f)(1) of the Water Resources 
Development Act of 1992 (106 Stat. 4835; 113 Stat. 335) is amended by 
striking ``$25,000,000 for''.
    (d) Paterson, Passaic Coun

In [ ]:
from google.colab import files
files.download('/content/tokenizer/tokenizer_config.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>